## The Training part of TNBC dataset, using XGBoost

In [ ]:
#coding=utf-8
import math
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import accuracy_score
import random
import pandas as pd
from database.entity_process import *
start_time = time.time()
subtype_to_num = {'BLIS':0, 'IM':1, 'LAR':2, 'MES':3}
def processed_TNBC_data():
  data_all = pd.read_excel('TNBC_new/all_module_complete.xlsx')
  name_list = list(data_all.keys())[3:]
  all_data = [[subtype_to_num[data_all.iloc[i, 2]], data_all.iloc[i, 3:]] for i in range(258)]
  return all_data, name_list
all_data, name_list = processed_TNBC_data()
random.seed(2022)
random.shuffle(all_data)

pro_list = pd.Index(list(name_list))
all_data = [[i[0], i[1][:]] for i in all_data]
all_data_pro_tv = all_data[:200]
tests = all_data[200:]


import csv
with open('./TNBC_new/temp_tv.csv','w') as file:
    fieldnames = ['label']
    fieldnames.extend(pro_list)
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for item in all_data_pro_tv:
        dat = {'label': item[0]}
        for i in range(len(item[1])):
            dat[pro_list[i]] = item[1][i]
        writer.writerow(dat)
with open('./TNBC_new/temp_test.csv','w') as file:
    fieldnames = ['label']
    fieldnames.extend(pro_list)
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for item in tests:
        dat = {'label': item[0]}
        for i in range(len(item[1])):
            dat[pro_list[i]] = item[1][i]
        writer.writerow(dat)
train = pd.read_csv("./TNBC_new/temp_tv.csv")
tests = pd.read_csv("./TNBC_new/temp_test.csv")
test_label=tests.iloc[:,0]
params={
    'booster':'gbtree',
    'objective':'multi:softmax',
    'num_class':4,
    'gamma':0.1,
    'max_depth':12,
    'lambda':0,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':3,
    'silent':0,
    'eta':0.007,
    'seed':1000,
    'nthread':7,
}
plst = list(params.items())
num_rounds = 50000
train_xy, val = train_test_split(train,test_size = 0.3,random_state=1)
y = train_xy.label
X = train_xy.drop(['label'],axis = 1)
val_y = val.label
val_X = val.drop(['label'],axis = 1)
xgb_val = xgb.DMatrix(val_X, label=val_y)
xgb_train = xgb.DMatrix(X, label=y)
y1 = tests.label
x1 = tests.drop(['label'],axis = 1)
xgb_test = xgb.DMatrix(x1, label=y1)
watchlist = [(xgb_train,'train'),(xgb_val, 'val')]
model = xgb.train(plst,xgb_train,num_rounds,watchlist,early_stopping_rounds=200)
model.save_model('./TNBC_new/xgb.model')
print('best best_ntree_limit',model.best_ntree_limit)
preds=model.predict(xgb_test,ntree_limit=model.best_ntree_limit)
accuracy_test=accuracy_score(preds,test_label)
print(f"the accuracy is {accuracy_test}.")
np.savetxt('./TNBC_new/xgb_submission.csv',np.c_[range(1,len(tests)+1),preds, test_label],delimiter=',',header='ImageId,Label,GroundTruth',comments='',fmt='%d')
cost_time=time.time()-start_time
print('success!','\n','cost time:',cost_time,'(s')

## The utilization of the trained model

#### The following part should repeat twice! to generate fmap

In [ ]:
#coding=utf-8
import math
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import accuracy_score
import random
import pandas as pd
from database.entity_process import *
start_time = time.time()
subtype_to_num = {'BLIS':0, 'IM':1, 'LAR':2, 'MES':3}
def processed_TNBC_data():
  data_all = pd.read_excel('TNBC_new/all_module_complete.xlsx')
  name_list = list(data_all.keys())[3:]
  all_data = [[subtype_to_num[data_all.iloc[i, 2]], data_all.iloc[i, 3:]] for i in range(258)]
  return all_data, name_list
all_data, name_list = processed_TNBC_data()
random.seed(2022)
random.shuffle(all_data)

pro_list = pd.Index(list(name_list))
all_data = [[i[0], i[1][:]] for i in all_data]
all_data_pro_tv = all_data[:200]
tests = all_data[200:]
features2 = [i.replace(" ", "-") for i in name_list]
import csv
with open('./TNBC_new/temp_tv.csv','w') as file:
    fieldnames = ['label']
    fieldnames.extend(pro_list)
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for item in all_data_pro_tv:
        dat = {'label': item[0]}
        for i in range(len(item[1])):
            dat[pro_list[i]] = item[1][i]
        writer.writerow(dat)
with open('./TNBC_new/temp_test.csv','w') as file:
    fieldnames = ['label']
    fieldnames.extend(pro_list)
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for item in tests:
        dat = {'label': item[0]}
        for i in range(len(item[1])):
            dat[pro_list[i]] = item[1][i]
        writer.writerow(dat)
train = pd.read_csv("./TNBC_new/temp_tv.csv")
tests = pd.read_csv("./TNBC_new/temp_test.csv")
test_label=tests.iloc[:,0]
params={
    'booster':'gbtree',
    'objective':'multi:softmax',
    'num_class':4,
    'gamma':0.1,
    'max_depth':12,
    'lambda':0,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':3,
    'silent':0,
    'eta':0.007,
    'seed':1000,
    'nthread':7,
}
plst = list(params.items())
num_rounds = 50000
train_xy, val = train_test_split(train,test_size = 0.3,random_state=1)
y = train_xy.label
X = train_xy.drop(['label'],axis = 1)
val_y = val.label
val_X = val.drop(['label'],axis = 1)
xgb_val = xgb.DMatrix(val_X, label=val_y)
xgb_train = xgb.DMatrix(X, label=y)
y1 = tests.label
x1 = tests.drop(['label'],axis = 1)
xgb_test = xgb.DMatrix(x1, label=y1)
watchlist = [(xgb_train,'train'),(xgb_val, 'val')]
f1 = open("./TNBC_new/xgb.model.fmap", "w") 
for i, feat in enumerate(features2):      
    f1.write('{0}\t{1}\tq\n'.format(i, feat)) 
import xgboost as xgb
from xgboost import plot_tree
import matplotlib.pyplot as plt
# load model.
bst = xgb.Booster()
bst.load_model('./TNBC_new/xgb.model')
model=bst
# # plot
plot_tree(bst, fmap='./TNBC_new/xgb.model.fmap',num_trees=0, rankdir='UT', ax=None)
plt.show()

#### The importance bar chart of XGBoost importance

In [ ]:
NUM = 3
graph = xgb.to_graphviz(bst, num_trees=NUM, **{'size': str(10)})
graph.render(filename='./TNBC/xgb_{}.dot'.format(NUM))
xgb.plot_importance(model, max_num_features=20, fmap='./TNBC_new/xgb.model.fmap')
model.dump_model("./TNBC_new/model.txt")
plt.savefig("./TNBC_new/xgb_importance.png")

#### Generation of ".graphml" file according to some constraint.

In [ ]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
imp = model.get_score(
            importance_type='gain', fmap='./TNBC_new/xgb.model.fmap')#weight/gain/cover
kk = list(imp.values())
import json
with open("TNBC_new/compound_reaction_complete.json", "r") as f:
  s = json.load(f)
names = [[i[2], i[0]] for i in s]
with open("TNBC_new/transcript_reaction_complete.json", "r") as f:
  s = json.load(f)
names += [[i[0], i[1]] for i in s]
names = [[i[0].replace("-", " "), i[1]] for i in names]
gene_ortholog = [i[2] for i in s]
name_dict = {}
for item in names:
  name_dict[item[0]] = item[1]

score_dict = {}

for item in imp.keys():
  item_2 = item.replace("-"," ")
  score_dict[name_dict[item_2]] = imp[item]
from neo4j import GraphDatabase
from py2neo import Node, Relationship, Path
import pandas as pd
import networkx as nx
from py2neo import Graph as pGraph
import py2neo
from matplotlib import pyplot as plt
from igraph import Graph as iGraph
import igraph
import networkx as nx
import time
import sys
THRESHOLD = 0.5
def graph_from_cypher(data):
    """Constructs a networkx graph from the results of a neo4j cypher query.
    Example of use:
    >>> result = session.run(query)
    >>> G = graph_from_cypher(result.data())

    Nodes have fields 'labels' (frozenset) and 'properties' (dicts). Node IDs correspond to the neo4j graph.
    Edges have fields 'type_' (string) denoting the type of relation, and 'properties' (dict)."""

    G = nx.MultiDiGraph()
    def add_node(node):
        # Adds node id it hasn't already been added
        u = node.identity
        if G.has_node(u):
            return
        G.add_node(u, labels=node._labels, properties=dict(node))

    def add_edge(relation):
        # Adds edge if it hasn't already been added.
        # Make sure the nodes at both ends are created
        for node in (relation.start_node, relation.end_node):
            add_node(node)
        # Check if edge already exists
        rel_type = str(type(relation))
        u = relation.start_node.identity
        v = relation.end_node.identity
        eid = relation.identity
        if G.has_edge(u, v, key=eid):
            return
        # If not, create it
        
        G.add_edge(u, v, key=eid, type_=rel_type[rel_type.find("data.")+5:-2], properties=dict(relation))

    def add_path(entry):
        for rel in entry.relationships:
            add_edge(rel)

    for d in data:
        for entry in d.values():
            # Parse node
            if isinstance(entry, Node):
                add_node(entry)

            # Parse link
            elif isinstance(entry, Relationship):
                add_edge(entry)
            elif isinstance(entry, Path):
                add_path(entry)
            else:
                raise TypeError("Unrecognized object")
    return G
start_time = time.time()
neo4j = pGraph("bolt://localhost:7687", auth=("neo4j", "libiao"))
all_available_kid = list(score_dict.keys())

dats = neo4j.run("match (n1:compound)-[p:substrate]-(n2:reaction)-[q:product]-(n3:compound),\
    (n2)-[r:enzyme]-(n4:ortholog)-[s:gene_expression]-(n5:gene) where n1.TNBC_valid = 1 and \
        n3.TNBC_valid = 1 and n5.kid in {} return *;".format(all_available_kid))

# dats = neo4j.run("match (n1:compound)-[p:substrate]-(n2:reaction)-[q:product]-(n3:compound),\
#     (n6:pathway)-[t:in_pathway]-(n2)-[r:enzyme]-(n4:ortholog)-[s:gene_expression]-(n5:gene) where n1.TNBC_valid = 1 and \
#         n3.TNBC_valid = 1 and n5.kid in {} and n6.kid in ['path:hsa00230'] return *;".format(all_available_kid))

end_time = time.time()
print("Data downloaded")
print(str(round((end_time-start_time)/60, 2))+ "min")
G = graph_from_cypher(dats.data())
print("Graph converted")
end_time = time.time()
print(str(round((end_time-start_time)/60, 2))+ "min")
import copy
gg = iGraph.from_networkx(G)
# EDGE_TYPE = ['expression', 'gene_expression', 'repression', 'activation', 'inhibition', 'substrate', 'product', 'enzyme', 'in_module', 'in_pathway']
EDGE_TYPE = ['expression', 'gene_expression', 'repression', 'activation', 'inhibition', 'substrate', 'product', 'enzyme', 'in_module',]
for edge in gg.es:
    if edge['type_'] not in EDGE_TYPE:
        gg.delete_edges(edge)
        pass
for vertice in gg.vs:
    if vertice.degree() == 0:
        gg.delete_vertices(vertice)
for vertice in gg.vs:
    if vertice.degree() == 0:
        gg.delete_vertices(vertice)
g = copy.deepcopy(gg)
g.delete_edges(g.es)
def adding_edge(e, g):
    if e not in g.es:
        g.add_edge(e.source_vertex, e.target_vertex, type_=e['type_'], properties=e['properties'])
cnt = 0

for vertice in gg.vs:
    v_type = list(vertice['labels'])[0]
    if v_type == "reaction":
        for e in vertice.all_edges():
            if e['type_'] in ['enzyme', 'substrate', 'product']:
            # if e['type_'] in ['enzyme', 'substrate', 'product', 'in_pathway']:
                adding_edge(e, g)
            elif e.source_vertex["properties"]['kid'] in all_available_kid + gene_ortholog:
                adding_edge(e, g)
    elif v_type == "gene":
        for e in vertice.all_edges():
            if e['type_'] in ['gene_expression']:
                adding_edge(e, g)

edge_clear = True
# for i in range(2):
#     for vertice in g.vs:
#         v_type = list(vertice['labels'])[0]
#         if vertice.degree() <= 2 and v_type == "reaction":
#             for e in vertice.all_edges():
#                 g.delete_edges(e)

# for vertice in g.vs:
#         v_type = list(vertice['labels'])[0]
#         if v_type == "compound":
#             if vertice["properties"]['kid'] not in all_available_kid + gene_ortholog:
#                 try:
#                     for e in vertice.all_edges():
#                         g.delete_edges(e)      
#                 except:
#                     pass
#                 g.delete_vertices(vertice)
# while True:
#     edge_clear = True
#     for vertice in g.vs:
#         v_type = list(vertice['labels'])[0]
#         if vertice.degree() == 0 and v_type == "reaction":
#             edge_clear = False
#             g.delete_vertices(vertice)
#         elif vertice.degree() == 0 and v_type in ["ortholog","gene",'compound','module']:
#             if vertice["properties"]['kid'] not in all_available_kid and vertice["properties"]['kid'] not in gene_ortholog:
#                 edge_clear = False
#                 g.delete_vertices(vertice)
#             else:
#                 edge_clear = False
#                 g.delete_vertices(vertice)
#     if edge_clear:
#         break
print("Final graph constructed")
end_time = time.time()
print(str(round((end_time-start_time)/60, 2))+ "min")
adj = g.get_adjacency()
SIDE = adj.shape[0]
avail = [True for i in range(SIDE)]
cnt = 0
comm_cnt = 0
community_list = []
community_list_sketch = []
import json
print("All done")
end_time = time.time()#'subGraph_light_'+str(THRESHOLD)+'_.graphml'
print(str(round((end_time-start_time)/60, 2))+ "min")
visual_style = {
# "vertex_size" : 5,
"bbox": (1000,1000),
"margin": 60,
"layout": "kk",
# "arrow_size": 1,
"edge_width": 0.3
}
node_labels_color = []
for i in g.vs["properties"]:
    try:
        y = score_dict[i['kid']]
        if y < 1.0:#weight=3.0, gain=1.0
            x = "black"
        elif y < 2.0:#weight=3.0, gain=2.0
            x = "green"
        else:
            x = "red"
    except:
        x = "grey"
    node_labels_color.append(x)
g.vs["label_color"] = node_labels_color
label_name = []
for i in g.vs["properties"]:
    if i['kid'] in all_available_kid:
        label_name.append(i['kid'] + "=" +str(round(score_dict[i['kid']], 2)))
        # label_name.append(name_dict_reverse[i['kid']] + "=" +str(round(score_dict[i['kid']], 2)))
    else:
        label_name.append(i['kid'])
g.vs["label"] = label_name#['_nx_name':<id>, 'labels':类别, 'properties', 'label']
col = {"compound":"orange", "reaction":"pink", "gene":"brown", "ortholog":"green", "module":"blue", "pathway": "red"}
siz = {"compound":15, "reaction":5, "gene":15, "ortholog":15, "module":15, "pathway":15}
g.vs["color"] = [col[list(i)[0]] for i in g.vs["labels"]]
g.vs["size"] = [siz[list(i)[0]] for i in g.vs["labels"]]
g.vs["label_dist"] = [1 for i in g.vs["labels"]]
g.vs["label_size"] = [6 for i in g.vs["labels"]]
# g.vs["label_color"] = ["blue" for i in g.vs["labels"]]
#[i['kid'] for i in g.vs["properties"]], [i['name'] for i in g.vs["properties"]]
#[list(i)[0] for i in g.vs["labels"]]
g.es['arrow_size'] = [0.5 for i in g.es['type_']]
e_col = {'expression':"green", 'gene_expression':"green", 'repression':"red", 
'activation':"green", 'inhibition':"red", 'substrate':"blue", 'product':"magenta", 'enzyme':"pink", "in_module":"orange", "in_pathway":"black"}
g.es['color'] = [e_col[i] for i in g.es['type_']]
g.es['label'] = [i for i in g.es['type_']]
g.es['label_color'] = [e_col[i] for i in g.es['type_']]
g.es["label_size"] = [6 for i in g.es["type_"]]

igraph.save(g, './TNBC_new/subGraph_light_2_'+str(THRESHOLD)+'_.graphml')

In [ ]:
li_imp = []
for vertice in g.vs:
  v_type = list(vertice['labels'])[0]
  if v_type in "reaction":
    num_nodes = 0
    importance_value = 0
    for e in vertice.all_edges():
      v_mid = e.source_vertex
      if list(v_mid['labels'])[0] == "compound":
        try:
          importance_value += score_dict[v_mid["properties"]['kid']]
          num_nodes += 1
        except:
          pass
      elif list(v_mid['labels'])[0] == "ortholog":
        for ed in v_mid.all_edges():
          v_end = ed.source_vertex
          if list(v_end['labels'])[0] == "gene":
            importance_value += score_dict[v_end["properties"]['kid']]
            num_nodes += 1
    if num_nodes == 0:
      num_nodes += 1
    li_imp.append([vertice["properties"]['kid'], num_nodes, importance_value, importance_value / num_nodes])
import math

li_imp.sort(key= lambda x: (x[3]+math.log(1),x[1]), reverse=True)
for i in li_imp:
  print(i) 
print([li_imp[i][0] for i in range(20)])

## SHAP Analysis

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
data = train_xy#train, train_xy, val, tests
cols = list(name_list)
shap_values = explainer.shap_values(data[cols])

#### Comprehesional analysis of shap values and display of the list of importance.
TYPE_NUMBER: {'BLIS':0, 'IM':1, 'LAR':2, 'MES':3}

ANALYSIS_TYPE : {0:absolute value of shap_values, 1: original shap_values}

In [ ]:
imp_0_list = []
TYPE_NUMBER = 0
ANALYSIS_TYPE = 0
for i in range(len(name_list)):
  if ANALYSIS_TYPE == 0:
    imp_0_list.append(
      [i, np.abs(shap_values[TYPE_NUMBER][:, i]).mean(), name_list[i]])
  elif ANALYSIS_TYPE == 1:
    imp_0_list.append([i, (shap_values[0][:,i]).mean(), name_list[i]])
  else:
    raise ValueError
imp_0_list.sort(key= lambda x: x[1], reverse=True)
# print([imp_0_list[i][1:] for i in range(20)])

# for i in range(500):
#   if np.abs(shap_values[0][:,i]).mean() > 0.0:
#     print(i, np.abs(shap_values[0][:,i]).mean(), name_list[i])
name_dict_reverse = {}
for item in name_dict.keys():
  name_dict_reverse[name_dict[item]] = item
shap_0_dict = {}
for item in imp_0_list:
  shap_0_dict[item[2]] = item[1]
li_imp = []
for vertice in g.vs:
  v_type = list(vertice['labels'])[0]
  if v_type in "reaction":
    num_nodes = 0
    importance_value = 0
    node_name = []
    node_imp = []
    for e in vertice.all_edges():
      v_mid = e.source_vertex
      if list(v_mid['labels'])[0] == "compound":
        try:
          importance_value += shap_0_dict[name_dict_reverse[v_mid["properties"]['kid']]]
          num_nodes += 1
          node_name.append(name_dict_reverse[v_mid["properties"]['kid']])
          node_imp.append(shap_0_dict[name_dict_reverse[v_mid["properties"]['kid']]])
        except:
          pass
      elif list(v_mid['labels'])[0] == "ortholog":
        for ed in v_mid.all_edges():
          v_end = ed.source_vertex
          if list(v_end['labels'])[0] == "gene":
            try:
              importance_value += shap_0_dict[name_dict_reverse[v_end["properties"]['kid']]]
              num_nodes += 1
              node_name.append(name_dict_reverse[v_end["properties"]['kid']])
              node_imp.append(shap_0_dict[name_dict_reverse[v_end["properties"]['kid']]])
            except:
              pass
    if num_nodes == 0:
      num_nodes += 1
    dat = neo4j.run("match (n:reaction)-[p]-(q:pathway) where n.kid in ['{}'] return q;".format(vertice["properties"]['kid']))
    li_imp.append([[vertice["properties"]['kid'], [nn['q']['kid'] for nn in list(dat)]],
       num_nodes, importance_value, importance_value / num_nodes, node_name, node_imp])
import math

# li_imp.sort(key= lambda x: (x[3]+math.log(1),x[1]), reverse=True)
li_imp.sort(key= lambda x: (x[3] * (x[1] > 2),x[1]), reverse=True)
def non_zero(list):
  num = 0
  eps = 1e-5
  for i in list:
    if abs(i) > eps:
      num += 1
  return num
# li_imp.sort(key= lambda x: (x[3] * (x[1] > 2) * (non_zero(x[5]) > 2),x[1]), reverse=True)

pathlist = []
path_dict = {}
over_pathlist = []
for item in li_imp:
  for name in item[0][1]:
    if pathlist.count(name) == 0:
      pathlist.append(name)
      path_dict[name] = [item[0][0]]
    else:
      path_dict[name].append(item[0][0])

for item in pathlist:
  dat = neo4j.run("match (n:gene)-[p]-(q:pathway) where q.kid in ['{}'] return n;".format(item))
  if len(list(dat)) > 400:
    over_pathlist.append(item)

global_over_path = ['path:hsa01100','path:hsa01110','path:hsa01120','path:hsa01210',
'path:hsa01212','path:hsa01230','path:hsa01232','path:hsa01250','path:hsa01240','path:hsa01220']
for i in global_over_path:
  if i not in over_pathlist:
    over_pathlist.append(i)  

for item in li_imp:
  left = []
  for name in item[0][1]:
    if name not in over_pathlist:
      left.append(name)
  item[0][1] = left

for i in li_imp:
  print(i) 
# print([li_imp[i][0] for i in range(20)])

#### SHAP Summary plot

In [ ]:
shap.summary_plot(shap_values, data[cols])